# 空間を扱うプログラミング

## §2 カメラ

コンピュータ上で3D空間を操作した後、その結果をスクリーン上に表示するためには、3Dから2Dへの座標変換、つまり投影変換が必要になります。この変換は主に以下の二つのステップに分けられます：

1. ワールド座標系からカメラ座標系への変換：
    * このステップでは、ワールド内に配置されたカメラの位置と姿勢を基に、3D空間がカメラからどのように見えるかを計算します。これにはカメラの「外部パラメータ」が使われます。
2. カメラ座標系から画像座標系への変換：
    * カメラが捉えた3Dの景色を2Dの画像上のピクセルに変換します。この計算にはカメラの「内部パラメータ」が使われます。

### §2-1 カメラ座標系と画像座標系

* カメラ座標系：
   * カメラ自身のローカル座標系とも言えます。Open3Dでは、カメラの視線方向を+Z軸、カメラの右方向を+X軸、カメラの下方向を+Y軸として設定しています。

<img alt="Camera coordinate system in Open3D" src="./notebook_assets/images/1_example_transformation.png" style="width:600px" />

* 画像座標系：
    * スクリーン上でピクセルによって表される座標系で、Open3Dではスクリーンの左上を原点とし、右方向を+X軸、下方向を+Y軸として定義しています。

<img alt="Screen coordinate system in Open3D" src="./notebook_assets/images/2_screen_coordinate_system.png" style="width:600px;" />


### §2-2 カメラモデル

カメラの特性を定義するためには、外部パラメータと内部パラメータの二つの主要な要素が重要です。

#### カメラ外部パラメータ

外部パラメータは、カメラ座標系におけるワールド座標原点の位置と姿勢として定義されます。これは次の行列 $\mathbf{E}$ で表されます：
(カメラからみたワールド座標軸の位置を $\mathbf{t}'$、 回転行列を $\mathbf{R}'$ (要素は $r_{ij}'$) とします)

$$
\mathbf{E} = \begin{bmatrix}
r'_{11} & r'_{12} & r'_{13} & t_x' \\
r'_{21} & r'_{22} & r'_{23} & t_y' \\
r'_{31} & r'_{32} & r'_{33} & t_z' \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

ワールド座標系におけるカメラの位置 $\mathbf{t}$ と姿勢を表す回転行列 $\mathbf{R}$ から、逆行列を使ってカメラ座標系での位置を計算できます：

$$
\mathbf{R}' = \mathbf{R}^T (回転行列の逆行列は、自身の転置行列に等しい) \\
\mathbf{t}' = - \mathbf{R}^T \mathbf{t}
$$

$\mathbf{E}$ を用いることでワールド座標系の任意の点 $\mathbf{p}$ のカメラ座標系での座標 $\mathbf{p}_C$ は以下のように計算されます：

$$
\mathbf{p}_C = \mathbf{E} \mathbf{p} \\
ただし \\
\mathbf{p} = \begin{bmatrix}
x \\ y \\ z \\ 1
\end{bmatrix} \\
\mathbf{p}_C = \begin{bmatrix}
x_C\\ y_C \\ z_C \\ 1
\end{bmatrix}
$$

#### カメラ内部パラメータ

内部パラメータは、3Dのカメラ座標系から2Dの画像座標系への変換を定義します。

このパラメータはカメラの位置や姿勢に依存しません。

ここでは最も単純なピンホールカメラモデルを定義します。


* ピンホールカメラの内部パラメータ：
    * 焦点距離 (focal length)：これはレンズの中心から焦点までの距離を表し、単位はピクセルです。理論上はレンズの焦点距離は一定ですが、カメラのイメージセンサーによっては、画素の縦横比が異なるため、スクリーンのX方向とY方向で焦点距離が異なる場合があります。
    * 主点 (principal point)：これは画像座標系で、レンズの光軸がスクリーンに交わる点の座標です。通常、スクリーンの中心（幅と高さの半分）を近似的に使用します。
* カメラ座標系から画像座標系への変換行列 (K 行列)：
    * 焦点距離をスクリーンのX方向とY方向でそれぞれ $f_X$, $f_Y$、主点を $(c_X, c_Y)$ としたとき、変換行列 $\mathbf{K}$ は次のように定義されます：

$$
\mathbf{K} = \begin{bmatrix}
f_X &   0 & c_X  \\
  0 & f_Y & c_Y  \\
  0 &   0 &   1  \\
\end{bmatrix}
$$


カメラ座標系の点 $\mathbf{p}_C$ から画像座標系の点 $\mathbf{p}_S$ への変換は次のように計算されます：

$$
\mathbf{p}_S = \mathbf{K} \frac{1}{z_C} \mathbf{p}_C \\
ただし \\
\mathbf{p}_C = \begin{bmatrix}
x_C \\ y_C \\ z_C
\end{bmatrix} \\
\mathbf{p}_S = \begin{bmatrix}
x_S\\ y_S \\ 1
\end{bmatrix}
$$

※上式のように、一般にカメラ座標系の点の x, y 座標を z 座標の値で割って規格化してからカメラ行列を乗じます。この操作は、空間の点をカメラの前方 1 単位の距離にあるスクリーン上に写すことに相当します。(このスクリーン上の座標を正規化画像座標系と呼びます)


これらのパラメータを用いることで、ワールド座標上の任意の点をスクリーン上に投影することが可能です。

* 実際のカメラレンズでは、スクリーン中心から離れるに従って像が歪むことを考慮します
* また、パノラマカメラや魚眼カメラのように広範囲を写すカメラレンズでは、ピンホールカメラモデルよりも複雑なカメラモデルを使用します

#### 投影変換

前のセクションで扱ったカメラ外部パラメータ・内部パラメータを組み合わせることで、ワールド座標上の任意の点 $\mathbf{p}$ の画像座標系での座標 $\mathbf{p}_S$ に変換できます。

この操作を投影変換と呼び、下記のように書けます: 

$$
\mathbf{p}_S = \mathbf{K} \mathbf{p}_C = \mathbf{K} \mathbf{E} \mathbf{p}
$$

ここで $\mathbf{K} \mathbf{E}$ で表される行列を投影行列と呼びます。

In [ ]:
# 必要なライブラリをインポート
from __future__ import annotations

import numpy as np
import open3d as o3d
import sympy as sp
from IPython.display import Math, display
from scipy.spatial.transform import Rotation

from util_lib.camera import PinholeCameraParameters, SimplePinholeCamera
from util_lib.transformable_object import TransformableObject
from util_lib.types import EulerOrder, Transform
from util_lib.visualization import draw_geometries
from util_lib.world import create_coordinate_objects

# ワールド・カメラ座標系の作成と読み込み
world_coordinate: o3d.geometry.Geometry = create_coordinate_objects()
original_camera = TransformableObject.load_model("./data/camera.gltf", enable_post_processing=True)


# 行列表示用の関数
def print_matrix(mat: np.ndarray | Transform) -> None:
    if isinstance(mat, Transform):
        mat_sym = sp.Matrix(mat.get_matrix())
    else:
        mat_sym = sp.Matrix(mat)
    mat_sym = mat_sym.applyfunc(lambda x: sp.Symbol(f"{x:.3f}"))
    display(Math(sp.latex(mat_sym)))


まず。初期配置されたカメラの視点を再現してみましょう。

* Open3D のカメラモデルはピンホールカメラモデルです。
* スクリーンサイズは FullHD (幅 1920 pixel、高さ 1080 px) とします
* カメラ内部パラメータとして下記を設定します。
  * 主点はスクリーンの中央とします。
  * 焦点距離は 600 px とします。 (なお、焦点距離を小さくすると一度に写る範囲が広くなります)

下記のように、カメラオブジェクトにくっついているローカル座標軸と、ワールド座標軸の一部が見えるはずです。
(見えなければズームアウトしてみてください)

<img alt="View of initial camera object" src="./notebook_assets/images/2_initial_camera_view.png" style="width:600px;" />

In [ ]:
# ピンホールカメラ内部パラメータ
image_size = (1920, 1080) # pixels (FullHD)
focal_length = 600 # pixels
principal_point = (image_size[0] / 2 - 0.5, image_size[1] / 2 - 0.5) # pixels
camera_parameters = PinholeCameraParameters(focal_length, principal_point, image_size)

# 初期カメラ姿勢の設定
# ワールドに初期配置されているカメラオブジェクトのローカル座標系と、カメラ座標系の位置と向きを合わせておく
initial_rotate = Rotation.from_euler(EulerOrder.xyz, [-90, 180, 0], degrees=True).as_matrix()
initial_pose = Transform.from_rotate_and_translate(initial_rotate, [0, 0, 0])
view_camera = SimplePinholeCamera(camera_parameters, initial_pose)

In [ ]:
print("K 行列 (k_11 ~ k_33 部分のみ)")
print_matrix(view_camera.get_intrinsic_matrix())
print("外部パラメータ")
print_matrix(view_camera.get_extrinsic_matrix())
draw_geometries(world_coordinate, original_camera, camera=view_camera)

### 任意の場所のカメラ視点を再現

ここでは位置がワールド座標系の (x, y, z) = (1, 1, 1)、姿勢が回転順序 y → x → z、回転角 (180°, -135°, 0°) の Euler 角で表されるようなカメラの視点を再現してみましょう。

<img alt="Example camera pose" src="./notebook_assets/images/2_example_camera_pose.png" style="width:600px;" />

<img alt="Example camera view" src="./notebook_assets/images/2_example_camera_view.png" style="width:600px;" />

In [ ]:
# カメラオブジェクトの移動と回転
camera_obj = original_camera.copy()
camera_obj.translate([1, 1, 1])
camera_obj.rotate_by_euler(EulerOrder.yxz, [180, -135, 0])

# ワールド座標系でカメラを移動・回転 (カメラオブジェクトから transformation 行列 を取得)
view = view_camera.copy()
transform = camera_obj.get_transform()
view.transform(transform)

print("ワールド座標系でのカメラオブジェクトの位置・姿勢")
print_matrix(transform)

print("カメラ外部パラメータ")
print_matrix(view.get_extrinsic_matrix())

print("カメラ外部パラメータ (Euler 角と並進ベクトル")
print_matrix(view.get_extrinsic_parameters(EulerOrder.xyz)[0])
print_matrix(view.get_extrinsic_parameters(EulerOrder.xyz)[1])

draw_geometries(world_coordinate, original_camera, camera_obj, camera=view)

### 外部パラメータの解釈

外部パラメータとは、カメラ座標系に対するワールド座標系の座標原点の位置・座標軸の姿勢を表すトランスフォーメーションでした。

#### カメラ座標系でのワールド座標系の解釈

* ワールド座標原点の位置：
    * カメラ座標系から見たとき、ワールド座標原点は $(X, Y, Z) = (-1, 0, \sqrt{2})$ に位置しています。これはカメラから見て左方向に1単位、カメラの正面方向に $\sqrt{2}$ 単位の位置にあることを意味します。
* ワールド座標軸の姿勢：
    * ワールド座標軸の姿勢は、カメラ座標系でのオイラー角 (X → Y → Z) で (-135°, 0, 0) の回転によって得られます。これはカメラ座標系において、ワールド座標軸がカメラの座標軸に比べて X 軸周りに -135°回転していることを示します。

以下の画像はこれらのパラメータを視覚的に示しています。最初の画像はカメラから見たワールド座標原点の位置を、二つ目の画像はワールド座標軸の姿勢を示しています。

<img alt="Example extrinsic parameter" src="./notebook_assets/images/2_camera_extrinsic_1.png" style="width:600px;" />
<img alt="Example extrinsic parameter" src="./notebook_assets/images/2_camera_extrinsic_2.png" style="width:600px;" />

このように、外部パラメータは3Dシーンをカメラを通してどのように捉えるかを決定する重要な要素です。